# Lecture 1: NumPy basics
In this first lecture we will cover the basic functionality of <a href="https://numpy.org/">NumPy</a>, the fundamental package for scientific computing with Python.

#### Attribution
This work is derived from
[Lecture-2-Numpy](https://github.com/jrjohansson/scientific-python-lectures/blob/master/Lecture-2-Numpy.ipynb)
by
[J. R. Johansson](http://jrjohansson.github.io/)
licensed under a [Creative Commons Attribution 3.0 Unported License](http://creativecommons.org/licenses/by/3.0/).



## Contents
- [Introduction](#introduction)
    - [Creating numpy arrays](#creating)
    - [File I/O](#fileio)
    - [Type casting](#casting)
- [Useful arrays](#generation)
- [General operations](#general)
    - [Accessing and modifying](#access)
    - [Reshaping and resizing](#reshaping)
    - [Stacking and repeating](#stacking)   
- [Mathematical operations](#math)
    - [Extracting data](#extraction)
    - [Arithmetic computations](#arithmetic)
    - [Data processing](#processing)
- [Iterating over array elements](#iterating)

#### Helper function

The function below will allow us to conveniently print data later on in this notebook. You should run the piece of code (but may skip its content) for now.

In [2]:
def head(filename, num=10):
    """Print the first lines of a text file.
    
    Arguments
    ---------
    filename : str
        Name of the text file.
    num : int
        Maximum number of lines to show (optional). Default value 10.
    """
    with open(filename) as file:
        for _, line in zip(range(num), file):
            print(line.rstrip())

## Introduction <a name="introduction"></a>

The `numpy` package (module) is used in almost all numerical computations using Python. It is a package that provides high-performance vector, matrix and higher-dimensional data structures for Python. It is implemented in C and Fortran so when calculations are vectorized (formulated with vectors and matrices), performance is very good. 

To use `numpy` you need to import the module. The common method to import `numpy` is as `np`, so that we only have to write 'np', instead of 'numpy', whenever we call a function from `numpy`.

In [3]:
import numpy as np

In the `numpy` package the terminology used for vectors, matrices and higher-dimensional data sets is *array*. 



### Creating `numpy` arrays <a name="creating"></a>
There are a number of ways to initialize new numpy arrays, for example from

* a Python list or tuples
* using functions that are dedicated to generating numpy arrays, such as `arange`, `linspace`, etc. (we will see those later).
* reading data from files

For example, to create new vector and matrix arrays from Python lists we can use the `numpy.array` function. Since we imported `numpy` as `np`, we use `np.array()`.

In [4]:
#Vector: the argument to the array function is a Python list
v = np.array([1,2,3,4])
print(v)

[1 2 3 4]


In [5]:
#Matrix: the argument to the array function is a nested Python list. Every element of the outer
#list is list corresponding to a row of the matrix.
M = np.array([[1, 2], [3, 4]])
print(M)

[[1 2]
 [3 4]]


The `v` and `M` objects are both of the type `ndarray` that the `numpy` module provides.

In [6]:
print(type(v))
print(type(M))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


The difference between the `v` and `M` arrays is only their shapes. We can get information about the shape of an array by using the `ndarray.shape` property.

In [7]:
print(v.shape)

(4,)


In [8]:
print(M.shape)

(2, 2)


The number of elements in an array `x`, which is simply the product of the numbers in `x.shape`, is available through the `ndarray.size` property:

In [9]:
print(M.size)

4


Equivalently, we could use the function `numpy.shape` and `numpy.size`

In [10]:
print(np.shape(M))

(2, 2)


In [11]:
print(np.size(M))

4


So far the `numpy.ndarray` looks awfully much like a Python list (or nested list). Why not simply use Python lists for computations instead of creating a new array type? 

There are several reasons:

* Python lists are very general. They can contain any kind of object. They are dynamically typed. They do not support mathematical functions such as matrix and dot multiplications. Implementing such functions for Python lists would not be very efficient because of the dynamic typing.
* Numpy arrays are **statically typed** and **homogeneous**. The type of the elements is determined when the array is created.
* Numpy arrays are memory efficient.
* Because of the static typing, fast implementation of mathematical functions such as multiplication and addition of `numpy` arrays can be implemented in a compiled language (C and Fortran is used).

Using the `dtype` (data type) property of an `ndarray`, we can see what type the data of an array has.

In [12]:
print(M.dtype)

int32


We get an error if we try to assign a value of the wrong type to an element in a numpy array:

In [13]:
#Use try-except to make sure that running the notebook doesn't stop here.
try:
    M[0, 0] = 'hello'
except Exception as err:
    print(err)

invalid literal for int() with base 10: 'hello'


If we want, we can explicitly define the type of the array data when we create it, using the `dtype` keyword argument: 

In [14]:
M = np.array([[1, 2], [3, 4]], dtype=int)
print('M = \n', M)
N = np.array([[1, 2], [3, 4]], dtype=float)
print('N = \n', N)
O = np.array([[1, 2], [3, 4]], dtype=complex)
print('O = \n', O)

M = 
 [[1 2]
 [3 4]]
N = 
 [[1. 2.]
 [3. 4.]]
O = 
 [[1.+0.j 2.+0.j]
 [3.+0.j 4.+0.j]]


Common data types that can be used with `dtype` are: `int`, `float`, `complex`, `bool`, `object`, etc.

We can also explicitly define the bit size of the data types, for example: `int64`, `int16`, `float128`, `complex128`. For example, `int64` allows us to define an integer variable in the range $\big[-2^{64}, \dots, 2^{64}\big]$.

Below we give some further properties for accessing (type) information of an array.

In [15]:
print(N.itemsize) # bytes per element

8


In [16]:
print(N.nbytes) # number of bytes

32


In [17]:
print(N.ndim) # number of dimensions

2


### File I/O using Comma-Separated Values (CSV) format <a name="fileio"></a>

A very common file format for data files is comma-separated values (CSV), or related formats such as TSV (tab-separated values). To read data from such files into Numpy arrays we can use the `numpy.loadtxt` function. We first show part of  *stockholm_td_adj.dat* using the `head` function we defined in the beginning. It displays temperature measurements throughout the year in Stockholm. The dataformat is: year, month, day, daily average temperature, low, high, location.

In [18]:
# Show the first lines of data file.
print(head('stockholm_td_adj.dat', 15))

1800  1  1    -6.1    -6.1    -6.1 1
1800  1  2   -15.4   -15.4   -15.4 1
1800  1  3   -15.0   -15.0   -15.0 1
1800  1  4   -19.3   -19.3   -19.3 1
1800  1  5   -16.8   -16.8   -16.8 1
1800  1  6   -11.4   -11.4   -11.4 1
1800  1  7    -7.6    -7.6    -7.6 1
1800  1  8    -7.1    -7.1    -7.1 1
1800  1  9   -10.1   -10.1   -10.1 1
1800  1 10    -9.5    -9.5    -9.5 1
1800  1 11    -6.4    -6.4    -6.4 1
1800  1 12    -5.8    -5.8    -5.8 1
1800  1 13    -4.4    -4.4    -4.4 1
1800  1 14    -4.4    -4.4    -4.4 1
1800  1 15    -4.4    -4.4    -4.4 1
None


We next load the data using `numpy.loadtxt`.

In [19]:
data = np.loadtxt('stockholm_td_adj.dat')

In [20]:
print(data.shape)

(77431, 7)


In [21]:
print(data[:15])

[[ 1.80e+03  1.00e+00  1.00e+00 -6.10e+00 -6.10e+00 -6.10e+00  1.00e+00]
 [ 1.80e+03  1.00e+00  2.00e+00 -1.54e+01 -1.54e+01 -1.54e+01  1.00e+00]
 [ 1.80e+03  1.00e+00  3.00e+00 -1.50e+01 -1.50e+01 -1.50e+01  1.00e+00]
 [ 1.80e+03  1.00e+00  4.00e+00 -1.93e+01 -1.93e+01 -1.93e+01  1.00e+00]
 [ 1.80e+03  1.00e+00  5.00e+00 -1.68e+01 -1.68e+01 -1.68e+01  1.00e+00]
 [ 1.80e+03  1.00e+00  6.00e+00 -1.14e+01 -1.14e+01 -1.14e+01  1.00e+00]
 [ 1.80e+03  1.00e+00  7.00e+00 -7.60e+00 -7.60e+00 -7.60e+00  1.00e+00]
 [ 1.80e+03  1.00e+00  8.00e+00 -7.10e+00 -7.10e+00 -7.10e+00  1.00e+00]
 [ 1.80e+03  1.00e+00  9.00e+00 -1.01e+01 -1.01e+01 -1.01e+01  1.00e+00]
 [ 1.80e+03  1.00e+00  1.00e+01 -9.50e+00 -9.50e+00 -9.50e+00  1.00e+00]
 [ 1.80e+03  1.00e+00  1.10e+01 -6.40e+00 -6.40e+00 -6.40e+00  1.00e+00]
 [ 1.80e+03  1.00e+00  1.20e+01 -5.80e+00 -5.80e+00 -5.80e+00  1.00e+00]
 [ 1.80e+03  1.00e+00  1.30e+01 -4.40e+00 -4.40e+00 -4.40e+00  1.00e+00]
 [ 1.80e+03  1.00e+00  1.40e+01 -4.40e+00 -4.40e+00

Alternatively, one can use `numpy.genfromtxt`. This function also allows you, e.g., to specify what to do with empty cells. The option `filling_values` lets you specify for every column one number that will be used to fill empty cells with.  The option `delimiter=';'` is added so that every row gets split based on semicolumns. (If you open the CSV-file with something like Notepad, on Windows, you will see that semicolumns are used to split the data.)

In [22]:
data_genfromtxt = np.genfromtxt('csv_data.csv', delimiter=';', filling_values=[10,20,30,40])
print(data_genfromtxt)

[[ 1.  2.  3.  4.]
 [ 5. 20. 30.  8.]
 [ 9. 10. 30. 40.]]


Using `numpy.savetxt` we can store a Numpy array to a file in CSV format. The following code should create a file `matrix.csv` in the folder where you stored this notebook.

In [23]:
M = np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]],dtype=int)
np.savetxt('matrix.csv', M)

In [24]:
print(M)

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]]


In [25]:
# Show how the file looks like.
print(head('matrix.csv'))

1.000000000000000000e+00 2.000000000000000000e+00 3.000000000000000000e+00 4.000000000000000000e+00
5.000000000000000000e+00 6.000000000000000000e+00 7.000000000000000000e+00 8.000000000000000000e+00
9.000000000000000000e+00 1.000000000000000000e+01 1.100000000000000000e+01 1.200000000000000000e+01
1.300000000000000000e+01 1.400000000000000000e+01 1.500000000000000000e+01 1.600000000000000000e+01
None


### Type casting <a name="casting"></a>

Since Numpy arrays are *statically typed*, the type of an array does not change once created. But we can explicitly cast an array of some type to another using the `astype` functions (see also the similar `asarray` function). This always create a new array of new type:

In [26]:
M = np.array([[1,2], [3,4]])

In [27]:
print(M.dtype)

int32


In [28]:
M2 = M.astype(float)
print(M2)

[[1. 2.]
 [3. 4.]]


In [29]:
print(M2.dtype)

float64


In [30]:
M3 = M.astype(bool)
print(M3)

[[ True  True]
 [ True  True]]


## Useful arrays <a name="generation"></a>

For larger arrays it is inpractical to initialize the data manually, using explicit python lists. Instead we can use one of the many functions in `numpy` that generate arrays of different forms. Some of the more common (that are particularly handy for plotting functions) are:

#### arange

In [31]:
# Create a range
x = np.arange(10)
print(x)

[0 1 2 3 4 5 6 7 8 9]


In [32]:
# Specify start, stop, step. Note that the stop-point is NOT included
x = np.arange(5, 15, 0.5) # arguments: start, stop, step
print(x)

[ 5.   5.5  6.   6.5  7.   7.5  8.   8.5  9.   9.5 10.  10.5 11.  11.5
 12.  12.5 13.  13.5 14.  14.5]


In [33]:
# Specify dtype
x = np.arange(10, dtype=float)
print(x)

[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]


In [34]:
x = np.arange(-1, 1, 0.1)
with np.printoptions(precision=3, suppress=True):
    print(x)

[-1.  -0.9 -0.8 -0.7 -0.6 -0.5 -0.4 -0.3 -0.2 -0.1 -0.   0.1  0.2  0.3
  0.4  0.5  0.6  0.7  0.8  0.9]


#### linspace

In [35]:
# Using linspace, BOTH end points are included. 
# The interval between two numbers is (stop - start)/(num-1)
x = np.linspace(0, 9, 26)  # start, stop, num. 
print(x)

[0.   0.36 0.72 1.08 1.44 1.8  2.16 2.52 2.88 3.24 3.6  3.96 4.32 4.68
 5.04 5.4  5.76 6.12 6.48 6.84 7.2  7.56 7.92 8.28 8.64 9.  ]


#### mgrid
This function can be used, e.g., to plot a 3D figure.

In [36]:
x, y = np.mgrid[0:5, 0:5] # similar to meshgrid in MATLAB

In [37]:
print(x)

[[0 0 0 0 0]
 [1 1 1 1 1]
 [2 2 2 2 2]
 [3 3 3 3 3]
 [4 4 4 4 4]]


In [38]:
print(y)

[[0 1 2 3 4]
 [0 1 2 3 4]
 [0 1 2 3 4]
 [0 1 2 3 4]
 [0 1 2 3 4]]


#### diag
This function can be used to quickly generate a matrix whose diagonal contains a desired vector $x$.

In [39]:
# a diagonal matrix
x = np.array([1,2,3])
D = np.diag(x)
print('D = \n', D)

D = 
 [[1 0 0]
 [0 2 0]
 [0 0 3]]


In [40]:
# diagonal with offset from the main diagonal
Y = np.diag([1,2,3,4], k=1) 
print('Y = \n', Y)
Z = np.diag([1,2,3,4], k=-2) 
print('Z = \n', Z)

Y = 
 [[0 1 0 0 0]
 [0 0 2 0 0]
 [0 0 0 3 0]
 [0 0 0 0 4]
 [0 0 0 0 0]]
Z = 
 [[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [1 0 0 0 0 0]
 [0 2 0 0 0 0]
 [0 0 3 0 0 0]
 [0 0 0 4 0 0]]


#### zeros and ones

In [41]:
x = np.zeros(4)
print(x)

[0. 0. 0. 0.]


In [42]:
x = np.zeros(4, dtype=int)
print(x)

[0 0 0 0]


In [43]:
X = np.zeros((3, 4)) #Note that when you define a matrix of zeros, you need to specify the dimensions in a tuple. 
                     #That is, np.zeros(3,3) is not correct
print(x)

[0 0 0 0]


In [44]:
X = np.ones((4, 3), dtype=int)
print(X)

[[1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]]


## General operations <a name="general"></a>

### Accessing and modifying <a name="access"></a>

### Indexing

We can index elements in an array using square brackets and indice. Unlike Matlab (as we will see later), in `numpy` **indexing starts at 0**, just like the regular Python `list`.

In [45]:
v = np.array([1,2,3,4])
M = np.array([[1,2,3,4], [5,6,7,8], [9,10,11,12]])

In [46]:
# v is a vector, and has only one dimension, taking one index
print(v[0])

1


In [47]:
# M is a matrix, or a 2 dimensional array, taking two indices 
print(M[1,1])

6


If we omit an index of a multidimensional array it returns the whole row (or, in general, a N-1 dimensional array) 

In [48]:
print(M)

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]


In [49]:
print(M[1])

[5 6 7 8]


The same thing can be achieved with using `:` instead of an index: 

In [50]:
print(M[1,:]) # row 1

[5 6 7 8]


In [51]:
print(M[:,0]) # column 0

[1 5 9]


Note that both `M[1, :]` and `M[:, 1]` return a one dimensional array (or vector in mathematical terminology). In *Matlab*, the analogous commands would return a row vector ($1 \times n$ double array) and a column vector ($n \times 1$ double array), respectively.

### Index slicing

Index slicing is the technical name for the syntax `M[lower:upper:step]` to extract part of an array:

In [52]:
M = np.array([1,2,3,4,5])
print(M)

[1 2 3 4 5]


In [53]:
print(M[1:3])  # note that upper is not included

[2 3]


We can omit any of the three parameters in `M[lower:upper:step]`:

In [54]:
print(M[::]) # lower, upper, step all take the default values

[1 2 3 4 5]


In [55]:
print(M[::2]) # step is 2, lower and upper defaults to the beginning and end of the array

[1 3 5]


In [56]:
print(M[:3]) # first three elements

[1 2 3]


In [57]:
print(M[3:]) # elements from index 3

[4 5]


In [58]:
print(M[::-1])  # reverse

[5 4 3 2 1]


Negative indices counts from the end of the array (positive index from the begining):

In [59]:
M = np.array([1,2,3,4,5])

In [60]:
print(M[-1]) # the last element in the array

5


In [61]:
print(M[-3:]) # the last three elements

[3 4 5]


Index slicing works exactly the same way for multidimensional arrays:

In [62]:
M = np.array([
    [
        n + 10 * m
        for n in range(5)
    ]
    for m in range(5)
])
print(M)

[[ 0  1  2  3  4]
 [10 11 12 13 14]
 [20 21 22 23 24]
 [30 31 32 33 34]
 [40 41 42 43 44]]


In [63]:
# a block from the original array
print(M[1:4, 1:4])

[[11 12 13]
 [21 22 23]
 [31 32 33]]


In [64]:
# strides
print(M[::2, ::2])

[[ 0  2  4]
 [20 22 24]
 [40 42 44]]


Some blocks cannot be specified using a slice. You can use the `ix_` function to obtain any sub block you need.

In [65]:
i = [0, 1]
j = [2, -1]  # -1 is last column
print(M[np.ix_(i, j)])

[[ 2  4]
 [12 14]]


Note that this is different from `M[i, j]`, which selects the entries in positions (0, 2) and (1, -1).

In [66]:
print(M[i, j])

[ 2 14]


### Assigning new values


We can assign new values to elements in an array using indexing. Array slices are *mutable*: if they are assigned a new value the original array from which the slice was extracted is modified:

In [67]:
M = np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]])
print(M)

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]]


In [68]:
M[0,3] = -1
print(M)

[[ 1  2  3 -1]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]]


In [69]:
M[1,[1,2]] = [-3,-10]
print(M)

[[  1   2   3  -1]
 [  5  -3 -10   8]
 [  9  10  11  12]
 [ 13  14  15  16]]


In [70]:
M[:,1] = np.arange(1,5)
print(M)

[[  1   1   3  -1]
 [  5   2 -10   8]
 [  9   3  11  12]
 [ 13   4  15  16]]


As the following examples show, we can also redefine rows and columns. Note that a `list` is automatically translated to a numpy array, and that scalars are *broadcasted* to the right size.

In [71]:
# replace first and second row by given array
M[1:3,:] = [2, 1, 2, 1]
print(M)

[[ 1  1  3 -1]
 [ 2  1  2  1]
 [ 2  1  2  1]
 [13  4 15 16]]


In [72]:
M[:,0] = -3  # all elements in 3rd column become -3
print(M)

[[-3  1  3 -1]
 [-3  1  2  1]
 [-3  1  2  1]
 [-3  4 15 16]]


### Fancy indexing

Fancy indexing is the name for when an array or list is used in-place of an index: 

In [73]:
row_indices = [1, 2, 3]
print(M[row_indices])

[[-3  1  2  1]
 [-3  1  2  1]
 [-3  4 15 16]]


In [74]:
col_indices = [1, 2, -1]  # remember, index -1 means the last element
print(M[row_indices, col_indices]) #Returns [M[1,1], M[2,2], M[3,-1]] 

[ 1  2 16]


We can also use index masks: If the index mask is an Numpy array of data type `bool`, then an element is selected (True) or not (False) depending on the value of the index mask at the position of each element: 

In [75]:
B = np.arange(5)
print(B)

[0 1 2 3 4]


In [76]:
row_mask = np.array([True, False, True, False, False])
print(B[row_mask])

[0 2]


In [77]:
# same thing
row_mask = np.array([1,0,1,0,0], dtype=bool)
print(B[row_mask])

[0 2]


This feature is very useful to conditionally select elements from an array, using for example comparison operators:

In [78]:
x = np.arange(0, 10, 0.5)
print(x)

[0.  0.5 1.  1.5 2.  2.5 3.  3.5 4.  4.5 5.  5.5 6.  6.5 7.  7.5 8.  8.5
 9.  9.5]


In [79]:
# values between 5 and 7.5
mask = (5 < x) & (x < 7.5)
print(mask)

[False False False False False False False False False False False  True
  True  True  True False False False False False]


In [80]:
print(x[mask])

[5.5 6.  6.5 7. ]


## Reshaping and resizing <a name="reshaping"></a>

The shape of an Numpy array can be modified **without copying** the underlaying data, which makes it a fast operation even for large arrays. These operations return a **view** of the original array. This means that changes in the original array are also applied to the reshaped array (and vice versa).

In [81]:
x = np.arange(24)
print(x)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]


Use the `reshape` function of array method `reshape` to get a reshaped view of the original array. In contrast to MATLAB, `numpy` fills the array using the *largest index changes fastest* principle, i.e., for two dimensions, the entries are filled along the second dimension (axis=1) first, and then along the first dimension (axis=0).

In [82]:
A = x.reshape(4, 6)
print(A)

[[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]
 [12 13 14 15 16 17]
 [18 19 20 21 22 23]]


In [83]:
A[1,:] = -1  ## Change all entries in second row.
print(A)

[[ 0  1  2  3  4  5]
 [-1 -1 -1 -1 -1 -1]
 [12 13 14 15 16 17]
 [18 19 20 21 22 23]]


In [84]:
# Corresponding values in x have also changed.
print(x)

[ 0  1  2  3  4  5 -1 -1 -1 -1 -1 -1 12 13 14 15 16 17 18 19 20 21 22 23]


You can get the transpose of an array using the `transpose` function or the `T` method. Both return a view of the original.

In [85]:
B = np.transpose(A)
print(B)

[[ 0 -1 12 18]
 [ 1 -1 13 19]
 [ 2 -1 14 20]
 [ 3 -1 15 21]
 [ 4 -1 16 22]
 [ 5 -1 17 23]]


In [86]:
print(A.T)

[[ 0 -1 12 18]
 [ 1 -1 13 19]
 [ 2 -1 14 20]
 [ 3 -1 15 21]
 [ 4 -1 16 22]
 [ 5 -1 17 23]]


You can also reshape an array using $-1$ as dimension. Then it adjusts the array automatically based on the second axis entered.

In [87]:
B = x.reshape(-1, 3)
print(B)

[[ 0  1  2]
 [ 3  4  5]
 [-1 -1 -1]
 [-1 -1 -1]
 [12 13 14]
 [15 16 17]
 [18 19 20]
 [21 22 23]]


We can also use the function `flatten` to make a higher-dimensional array into a vector. But this function creates a **copy** of the data. This means that changes to the original array are not applied to the copy (and vice versa).

In [88]:
B = A.flatten()
print(B)

[ 0  1  2  3  4  5 -1 -1 -1 -1 -1 -1 12 13 14 15 16 17 18 19 20 21 22 23]


In [89]:
B[0:5] = 10
print(B)

[10 10 10 10 10  5 -1 -1 -1 -1 -1 -1 12 13 14 15 16 17 18 19 20 21 22 23]


In [90]:
print(A) # now A has not changed, because B's data is a copy of A's, not refering to the same data

[[ 0  1  2  3  4  5]
 [-1 -1 -1 -1 -1 -1]
 [12 13 14 15 16 17]
 [18 19 20 21 22 23]]


Note that `flatten()` flattens the rows of $A$ into a one-dimensional arrays. Alternatively, we can also do this along the columns, using the option 'F' (Fortran-style).

In [91]:
print('A = \n',A)
D = A.flatten('F')
print('D = \n',D)

A = 
 [[ 0  1  2  3  4  5]
 [-1 -1 -1 -1 -1 -1]
 [12 13 14 15 16 17]
 [18 19 20 21 22 23]]
D = 
 [ 0 -1 12 18  1 -1 13 19  2 -1 14 20  3 -1 15 21  4 -1 16 22  5 -1 17 23]


You can also use the `ravel` method to turn a higher-dimensional array into a vector. However, in contrast to `flatten`, `ravel` returns a **view** of the original data.

In [92]:
C = A.ravel()
print(C)

[ 0  1  2  3  4  5 -1 -1 -1 -1 -1 -1 12 13 14 15 16 17 18 19 20 21 22 23]


In [93]:
C[:5] = 20
print(C)

[20 20 20 20 20  5 -1 -1 -1 -1 -1 -1 12 13 14 15 16 17 18 19 20 21 22 23]


In [94]:
# Values in A have also changed.
print(A)

[[20 20 20 20 20  5]
 [-1 -1 -1 -1 -1 -1]
 [12 13 14 15 16 17]
 [18 19 20 21 22 23]]


If you select a sub array, then this is also a **view** of the corresponding entries in the original array.

In [95]:
v = A[:, 0]  ## View of the first column of A
print(v)

[20 -1 12 18]


In [96]:
v[3] = 100  # also changes corresponding element of A

In [97]:
print(A)

[[ 20  20  20  20  20   5]
 [ -1  -1  -1  -1  -1  -1]
 [ 12  13  14  15  16  17]
 [100  19  20  21  22  23]]


If you want to work with a **copy** of the original array to prevent modifying the original values, then you can use the `copy` method.

In [98]:
print(A)

[[ 20  20  20  20  20   5]
 [ -1  -1  -1  -1  -1  -1]
 [ 12  13  14  15  16  17]
 [100  19  20  21  22  23]]


In [99]:
B = A.copy()

In [101]:
B[1, 1] = 1000
print('A = \n',A)  # not modified
print('B = \n',B)

A = 
 [[ 20  20  20  20  20   5]
 [ -1  -1  -1  -1  -1  -1]
 [ 12  13  14  15  16  17]
 [100  19  20  21  22  23]]
B = 
 [[  20   20   20   20   20    5]
 [  -1 1000   -1   -1   -1   -1]
 [  12   13   14   15   16   17]
 [ 100   19   20   21   22   23]]


 ## Stacking and repeating <a name="stacking"></a>

Using function `repeat`, `tile`, `stack`, `vstack`, `hstack`, and `concatenate` we can create larger vectors and matrices from smaller ones. If you are not familiar with these functions, then you often don't get the results you want. Look at the official documentation for more information and examples.

### tile and repeat

In [10]:
a = np.array([[1, 2], [3, 4]])
print(a)

[[1 2]
 [3 4]]


In [11]:
# repeat each element 3 times
b = np.repeat(a, 3)
print(b)

[1 1 1 2 2 2 3 3 3 4 4 4]


In [12]:
# tile the matrix 3 times 
c = np.tile(a, 3)
print(c)

[[1 2 1 2 1 2]
 [3 4 3 4 3 4]]


### concatenate
Arrays need to have the same dimensions in order to concatenate them.

In [13]:
b = np.array([[5, 6]])  # b has shape (1, 2) not (2,)
print(b)

[[5 6]]


In [14]:
C = np.concatenate((a, b), axis=0)
print(C)

[[1 2]
 [3 4]
 [5 6]]


In [15]:
D = np.concatenate((a, b.T), axis=1)
print(D)

[[1 2 5]
 [3 4 6]]


### hstack and vstack
`np.vstack` does the same as `np.concatenate`, but $c$ below can have dimension $(2,)$, instead of $(1,2)$ as was needed for $b$ above.

In [16]:
c = np.array([7,8])
D = np.vstack((a, c))
print(D)

[[1 2]
 [3 4]
 [7 8]]


For `np.hstack` one needs the correct dimensions again.

In [17]:
d = np.array([[7,8]])
D = np.hstack((a, d.T))
print(D)

[[1 2 7]
 [3 4 8]]


### Concatenation using `np.c_`

The `np.r_` and `np.c_` methods are a bit obscure and its documentation is hard to follow. However, these functions are also very convenient to concatenate arrays. Sometimes this is much more convenient than using `concatenate`.

In [18]:
print(a)

[[1 2]
 [3 4]]


In [19]:
x = np.array([7,8])

In [20]:
Y = np.c_[a, x]
print(Y)

[[1 2 7]
 [3 4 8]]


This cannot be accomplished using `concatenate` without first reshaping ``x`` to `x[:,None]`.

In [21]:
try:
    print(np.concatenate((a, x), axis=1))
except Exception as err:
    print(err)

all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)


In [22]:
Z = np.concatenate((a, x[:, None]), axis=1)
print(Z)

[[1 2 7]
 [3 4 8]]


#### About [:,None]
So what does it mean to 'reshape' ``x`` to `x[:,None]`? If we define `x = np.array([7,8])` then `x` has shape (2,), that is, it only has one dimension of size 2, whereas `a` has two dimensions of size (2,2). You can check this using `np.shape()`.

In [31]:
print(np.shape(x))
print(np.shape(a))

(2,)
(2, 2)


In order to use `np.concatenate`, both vectors need to have the same number of dimensions, which means we want to make sure `x` has two dimensions as well. This can be accomplished by changing `x` to `x[:,None]`, which changes the dimension from (2,) to (2,1). Then `a` has dimensions (2,2) and `x` (2,1), which means they have the right shapes in order to apply `np.concatenate`.

In [33]:
print('x = \n',x)
print('Dimension of x is \n', np.shape(x))
print('x[:,None] = \n', x[:,None])
print('Dimension of x[:,None] is \n', np.shape(x[:,None])) #Now x has dimension (2,1), i.e., x has two dimensions. 

x = 
 [7 8]
Dimension of x is 
 (2,)
x[:,None] = 
 [[7]
 [8]]
Dimension of x[:,None] is 
 (2, 1)


The use of `np.r_` is a bit more obscure. Here we have to specify that we stack the rows (the $0$-th dimension), and that the resulting array is two-dimensional. (For now, I would not recommend to use this function, but this is just so that you are aware of it.)

In [159]:
Z = np.r_['0,2',a,x]
print(Z)

[[1 2]
 [3 4]
 [7 8]]


## Mathematical operations <a name="math"></a>

## Extracting data <a name="extraction"></a>

### nonzero

The index mask can be converted to position index using the `nonzero` function. `np.nonzero` returns a tuple containing an array and dtype.

In [160]:
x = np.arange(0,10,0.5)
mask = (5 < x) & (x < 7.5)
indices = np.nonzero(mask) 
print(indices)
print(indices[0])

(array([11, 12, 13, 14], dtype=int64),)
[11 12 13 14]


In [161]:
y = x[indices] # this indexing is equivalent to the fancy indexing x[mask]
print(y)

[5.5 6.  6.5 7. ]


### where

The same thing can be accomplished using the `where` function. However, the `where` function can also be used as kind of if-else construction.

In [165]:
print(x)

[0.  0.5 1.  1.5 2.  2.5 3.  3.5 4.  4.5 5.  5.5 6.  6.5 7.  7.5 8.  8.5
 9.  9.5]


In [166]:
# Without additional arguments, returns the same as `nonzero`.
y = np.where(x > 5)
print(y)

(array([11, 12, 13, 14, 15, 16, 17, 18, 19], dtype=int64),)


In [167]:
# If x > 5, then return -10, else return 20.
z = np.where(x > 5, -10, 20)
print(z)

[ 20  20  20  20  20  20  20  20  20  20  20 -10 -10 -10 -10 -10 -10 -10
 -10 -10]


### diag

With the diag function we can also extract the diagonal and subdiagonals of an array. (Remember that `np.diag` applied to a one-dimensional array $x$ returns a matrix whose diagonal is $x$.)

In [102]:
M = np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]])
print('M = \n',M)
D = np.diag(M)
print('D = \n',D)

M = 
 [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]]
D = 
 [ 1  6 11 16]


In [103]:
d = np.diag(M, -1)  # one below main diagonal
print(d)

[ 5 10 15]


## Arithmetic computations <a name="arithmetic"></a>

Vectorizing code is the key to writing efficient numerical calculation with Python/Numpy. That means that as much as possible of a program should be formulated in terms of matrix and vector operations, like matrix-matrix multiplication.

### Scalar-array operations

We can use the usual arithmetic operators to multiply, add, subtract, and divide arrays with scalar numbers.

In [119]:
v = np.arange(0, 4)
M = np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]])

In [120]:
print(v * 2)

[0 2 4 6]


In [121]:
print(v + 2)

[2 3 4 5]


In [122]:
print(M * 2) #Multiplies every entry with 2

[[ 2  4  6  8]
 [10 12 14 16]
 [18 20 22 24]
 [26 28 30 32]]


In [123]:
print(M + 2) #Add 2 two to every element of M

[[ 3  4  5  6]
 [ 7  8  9 10]
 [11 12 13 14]
 [15 16 17 18]]


In [124]:
print(M / 2) #Divides every element of M by two.

[[0.5 1.  1.5 2. ]
 [2.5 3.  3.5 4. ]
 [4.5 5.  5.5 6. ]
 [6.5 7.  7.5 8. ]]


### Element-wise array-array operations

When we add, subtract, multiply and divide arrays with each other, the default behaviour is **element-wise** operations. 

In [125]:
print(M * M)  # element-wise multiplication

[[  1   4   9  16]
 [ 25  36  49  64]
 [ 81 100 121 144]
 [169 196 225 256]]


In [126]:
print(v * v)

[0 1 4 9]


### Broadcasting

Broadcasting rules allow us to conveniently perform operations with multiple arrays of different dimensions. If a dimension is "missing" or has dimension 1 along an axis, then the dimension can be created or duplicated to match other arrays.

In [127]:
print(v)

[0 1 2 3]


In [128]:
w = np.array([[10], [20], [30]])
print(w)

[[10]
 [20]
 [30]]


In [129]:
print(v.shape)
print(w.shape)

(4,)
(3, 1)


In [130]:
V = v + w
print(V)

[[10 11 12 13]
 [20 21 22 23]
 [30 31 32 33]]


In [131]:
#Every entry in first column gets muliplied with v1[0], in second column with v1[1], etc.. 
print(M * v) 

[[ 0  2  6 12]
 [ 0  6 14 24]
 [ 0 10 22 36]
 [ 0 14 30 48]]


In [133]:
#Every entry in first row gets muliplied with v1[0], in second row with v1[1], etc.
print(M * v[:,None])

[[ 0  0  0  0]
 [ 5  6  7  8]
 [18 20 22 24]
 [39 42 45 48]]


## Data processing <a name="processing"></a>

Often it is useful to store datasets in Numpy arrays. Numpy provides a number of functions to calculate statistics (like mean, standard deviation, maximum, minimum, etc...) of datasets in arrays. 

For example, let's calculate some properties from the Stockholm temperature dataset that was imported in the introduction (File I/O section).

In [137]:
#Remember that we stored the temperature in the 'data' variable:
print(head('stockholm_td_adj.dat', 15))
print('Dimensions of data set are \n', data.shape)

1800  1  1    -6.1    -6.1    -6.1 1
1800  1  2   -15.4   -15.4   -15.4 1
1800  1  3   -15.0   -15.0   -15.0 1
1800  1  4   -19.3   -19.3   -19.3 1
1800  1  5   -16.8   -16.8   -16.8 1
1800  1  6   -11.4   -11.4   -11.4 1
1800  1  7    -7.6    -7.6    -7.6 1
1800  1  8    -7.1    -7.1    -7.1 1
1800  1  9   -10.1   -10.1   -10.1 1
1800  1 10    -9.5    -9.5    -9.5 1
1800  1 11    -6.4    -6.4    -6.4 1
1800  1 12    -5.8    -5.8    -5.8 1
1800  1 13    -4.4    -4.4    -4.4 1
1800  1 14    -4.4    -4.4    -4.4 1
1800  1 15    -4.4    -4.4    -4.4 1
None
Dimensions of data set are 
 (77431, 7)


#### mean

Compute the mean by using the `mean` function, or the `mean` method of an `array` object.

In [138]:
# the temperature data is in column 3
print(np.mean(data[:,3]))

6.197109684751585


In [139]:
# Or use the array method
print(data[:, 3].mean())

6.197109684751585


The daily mean temperature in Stockholm over the last 200 years has been about 6.2 C.

#### standard deviations and variance

By default, `std` and `var` return the **population** standard deviation and variance functions, respectively. If you want the **sample** standard deviation or variance, then you should specify the optional `ddof` input argument.

In [140]:
print('Standard deviation is \n',np.std(data[:,3]))
print('Variance is \n', np.var(data[:,3]))

Standard deviation is 
 8.282271621340573
Variance is 
 68.59602320966341


In [141]:
print('Sample standard deviation is \n',np.std(data[:, 3], ddof=1))  ## sample stdev (divide by n-1)

Sample standard deviation is 
 8.282325103484965


#### min and max

In [147]:
# lowest daily average temperature
print(data[:,3].min())

-25.8


In [148]:
# highest daily average temperature
print(data[:,3].max())

28.3


### Computations on subsets of arrays

We can compute with subsets of the data in an array using indexing, fancy indexing, and the other methods of extracting data from an array (described above).

For example, let's go back to the temperature dataset:

In [146]:
print(head('stockholm_td_adj.dat', 5))

1800  1  1    -6.1    -6.1    -6.1 1
1800  1  2   -15.4   -15.4   -15.4 1
1800  1  3   -15.0   -15.0   -15.0 1
1800  1  4   -19.3   -19.3   -19.3 1
1800  1  5   -16.8   -16.8   -16.8 1
None


The dataformat is: year, month, day, daily average temperature, low, high, location.

If we are interested in the average temperature only in a particular month, say February, then we can create a index mask and use it to select only the data for that month using:

In [151]:
print(np.unique(data[:,1])) # the month column takes values from 1 to 12

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]


In [152]:
mask_feb = data[:,1] == 2
print(mask_feb[30:60]) #Day 31 to 58 represent the first occurence of the month February in the data

[False  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True False]


In [155]:
# Total observations and observations in February
print('Total number of observations is \n', len(data))
print('Total number of observations in February is', sum(mask_feb))

Total number of observations is 
 77431
Total number of observations in February is 5987


In [156]:
# the temperature data is in column 3
print(np.mean(data[mask_feb, 3]))

-3.212109570736596


### Calculations with higher-dimensional data

When functions such as `min`, `max`, etc. are applied to a multidimensional arrays, it is sometimes useful to apply the calculation to the entire array, and sometimes only on a row or column basis. Using the `axis` argument we can specify how these functions should behave: 

In [157]:
m = np.random.rand(3,3)
print(m)

[[0.07236081 0.99871295 0.14574843]
 [0.11255882 0.73346864 0.18474869]
 [0.26792826 0.27473646 0.06227972]]


In [158]:
# global max
print(m.max())

0.998712947952901


In [159]:
# max in each column
print(m.max(axis=0))

[0.26792826 0.99871295 0.18474869]


In [160]:
# max in each row
print(m.max(axis=1))

[0.99871295 0.73346864 0.27473646]


Many other functions and methods in the `array` class accept the same (optional) `axis` keyword argument.

To compute the maximum or minimum over several arguments, use the `fmax` or `fmin` functions, respectively.

In [161]:
v = np.random.rand(4,4)
print('v = \n', v)
w = np.random.rand(4,4)
print('w = \n', w)

v = 
 [[0.45843646 0.03708026 0.05789331 0.71153991]
 [0.10902846 0.89195853 0.20537997 0.43293587]
 [0.32032418 0.93339607 0.6171293  0.9733007 ]
 [0.30324695 0.03969654 0.98968864 0.10293991]]
w = 
 [[0.20585622 0.25548931 0.0365698  0.41920309]
 [0.74545615 0.34159365 0.20264315 0.67901439]
 [0.86434066 0.347554   0.78644695 0.14966872]
 [0.05903573 0.01419319 0.75968848 0.02255779]]


In [162]:
print(np.fmax(v,w)) #Gives pointwise maximum

[[0.45843646 0.25548931 0.05789331 0.71153991]
 [0.74545615 0.89195853 0.20537997 0.67901439]
 [0.86434066 0.93339607 0.78644695 0.9733007 ]
 [0.30324695 0.03969654 0.98968864 0.10293991]]


In [163]:
print(np.fmin(v,w)) #Gives pointwise minimum

[[0.20585622 0.03708026 0.0365698  0.41920309]
 [0.10902846 0.34159365 0.20264315 0.43293587]
 [0.32032418 0.347554   0.6171293  0.14966872]
 [0.05903573 0.01419319 0.75968848 0.02255779]]


## Iterating over array elements <a name="iterating"></a>

Generally, we want to avoid iterating over the elements of arrays whenever we can (at all costs). The reason is that in a interpreted language like Python (or MATLAB), iterations are really slow compared to vectorized operations. 

However, sometimes iterations are unavoidable. For such cases, the Python `for` loop is the most convenient way to iterate over an array:

In [164]:
v = np.array([1,2,3,4])

for element in v:
    print(element)

1
2
3
4


In [165]:
M = np.array([[1,2], [3,4]])

for row in M:
    print('row:', row)
    
    for element in row:
        print(element)

row: [1 2]
1
2
row: [3 4]
3
4


When we need to iterate over each element of an array and modify its elements, it is convenient to use the `enumerate` function to obtain both the element and its index in the `for` loop: 

In [166]:
for row_idx, row in enumerate(M):
    print('row_idx', row_idx, 'row', row)
    
    for col_idx, element in enumerate(row):
        print('col_idx', col_idx, 'element', element)
       
        # update the matrix M: square each element
        M[row_idx, col_idx] = element ** 2

row_idx 0 row [1 2]
col_idx 0 element 1
col_idx 1 element 2
row_idx 1 row [3 4]
col_idx 0 element 3
col_idx 1 element 4


In [168]:
# each element in M is now squared
print(M)

[[ 1  4]
 [ 9 16]]
